In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys 
sys.path.append("../")

In [3]:
from RNN_utils import get_deep_rnn, chars_to_one_hot, sample
import numpy as np

In [4]:
model=get_deep_rnn((100, 69), dense_units=69, stateful=True)

input shape =  (100, 69)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 100, 200)             216000    
_________________________________________________________________
lstm_2 (LSTM)                (1, 200)                  320800    
_________________________________________________________________
dense (Dense)                (1, 69)                   13869     
Total params: 550,669
Trainable params: 550,669
Non-trainable params: 0
_________________________________________________________________


## Cuanto vale T y cuanto vale D?

In [5]:
layer1 = model.get_layer('lstm_1')
layer1.get_config()['batch_input_shape']

(1, 100, 69)

In [6]:
chars = np.load('training_chars.npy')
chars_to_indices = dict((c, i) for i, c in enumerate(chars))
indices_to_chars = dict((i, c) for i, c in enumerate(chars))

In [7]:
window = 100

In [9]:
# model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')
model.load_weights('bodes_sangre_LSTM_deep_dropout_05.hdf5')

In [10]:
def predict_phrase(model, initial_seed, window = 100, verbose=True, sample_type = 'greedy'):
    # El numeral no esta en el diccionario por lo que lo pasa a todos ceros en el one-hot
    initial_seed=initial_seed[-window:]
    initial_text = '#'*(window-len(initial_seed))+initial_seed # Padding
    X_test = chars_to_one_hot(initial_seed, chars, chars_to_indices, 100)
    probs = model.predict(X_test)
    probs_sorted_indexes = np.argsort(probs)[0][::-1]
    if verbose:
        print([(indices_to_chars[i], probs[0][i]) for i in probs_sorted_indexes[:5]])
    return probs[0]

In [11]:
def sample_probs(probs, sample_type='greedy', temp=1):
    if (sample_type=='greedy'):
        return np.argmax(probs)
    if (sample_type=='simple'):
        return sample(probs, temp, verbose=False, return_dist=True)

## Distribución de probabilidades para:
#### "Estaba el novi"

In [48]:
phrase = "Estaba el novi"
probs = predict_phrase(model, phrase)

[('o', 0.94839877), ('a', 0.040401522), ('e', 0.004838877), ('l', 0.0019398334), ('t', 0.0013715479)]


In [49]:
indices_to_chars[sample_probs(probs, sample_type='greedy')]

'o'

In [56]:
temp = 10
print(phrase)
print('Temperatura:', temp)
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print('Prob dist:', list(zip(dist[sorted_idxs][:5], chars[sorted_idxs][:5])))
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print('sample:', samples)

Estaba el novi
Temperatura: 10
Prob dist: [(0.049700554, 'o'), (0.03624949, 'a'), (0.029318152, 'e'), (0.02675707, 'l'), (0.0258454, 't')]
sample: ['d', 'v', 'a', 'I', 'v', 'u', 'O', 'a', ' ', '(', 'h', 'p', 'o', 'd', '(', 'é', ':', 't', 'n', 'é']


## Distribución de probabilidades para:
#### "Estaba la novi"

In [40]:
probs = predict_phrase(model, "Estaba la novi")
indices_to_chars[sample_probs(probs, sample_type='greedy')]

[('a', 0.9812135), ('o', 0.01519752), ('e', 0.0021953336), ('l', 0.0005232005), ('i', 0.00018333018)]


'a'

### Mismo ejemplo pero cambiar temperatura de ruido
Notar que muestramos (Cambiar temperatura)

In [42]:
temp = 2
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(list(zip(dist[sorted_idxs][:5], chars[sorted_idxs][:5])))
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[(0.75674635, 'a'), (0.09417919, 'o'), (0.03579471, 'e'), (0.017474415, 'l'), (0.01034393, 'i')]
['a', 'a', 'e', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'e', 'o', 'a', 'a', 'i']


## Distribución de probabilidades para:
#### "Estaba la novia"

In [23]:
probs = predict_phrase(model, "Estaba la novia")
indices_to_chars[sample_probs(probs, sample_type='greedy')]

[('.', 0.66570485), (' ', 0.21843682), (',', 0.04832371), ('\n', 0.043989368), ('!', 0.0070391377)]


'.'

### Mismo ejemplo pero cambiar temperatura de ruido
Notar que muestramos

In [38]:
temp = 1
index, dist = sample(probs, temp, verbose=False, return_dist=True)
sorted_idxs = np.argsort(dist)[::-1]
print(list(zip(dist[sorted_idxs][:5], chars[sorted_idxs][:5])))
samples = []
for i in range(20):
    index, dist = sample_probs(probs, sample_type='simple', temp=temp)
    samples.append(indices_to_chars[index])
print(samples)

[(0.66570485, '.'), (0.21843682, ' '), (0.04832371, ','), (0.043989368, '\n'), (0.0070391377, '!')]
['.', '.', ' ', '.', '.', '.', '.', '.', '.', '.', '\n', '.', '.', '.', '\n', '.', '\n', '.', '.', ' ']


## Generamos texto
### Greedy Search

In [57]:
def greedy_search(model, phrase, N=200, window=100):
    print(phrase, end='')
    for i in range(N):
        probs = predict_phrase(model, phrase, verbose=False)
        next_char = indices_to_chars[sample_probs(probs, sample_type='greedy')]
        phrase=phrase + next_char
        print(next_char, end='')

In [58]:
text = open('bodas_de_sangre.txt').read()

In [59]:
print(text[:100])

BODAS DE SANGRE(1933)

Poema trágico en tres actos y siete cuadros

Personajes
 
Madre
Criada
Leonar


In [60]:
greedy_search(model, text[:100])

BODAS DE SANGRE(1933)

Poema trágico en tres actos y siete cuadros

Personajes
 
Madre
Criada
Leonardo
y el agua pasa a la novia.
MUCHACHA 1:
¡Ay caballo se alegre
con el agua!
MUJER:
¡Ay caballo se pone a la boda.
MUCHACHA 1:
¡Ay caballo se alegre
con el agua!
MUJER:
¡Ay caballo se pone a la boda.


In [61]:
greedy_search(model, 'NOVIA:')

NOVIA: (En la boda) ¡Ay!

NOVIO: (A la novia) ¿Qué te acabar que la casa?

NOVIO: (A la novia) ¿Qué te acabar que la casa?

NOVIO: (A la novia) ¿Qué te acabar que la casa?

NOVIO: (A la novia) ¿Qué te acaba

In [63]:
greedy_search(model, 'JULIAN:', 300)

JULIAN: malen de la boda.

NOVIO: (Alegre) ¡Ay!

MADRE: ¿Qué te pasa?

NOVIO: ¿Está contenta?

NOVIO: ¿Está con el agua!

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Qué está a la novia?

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Qué está a la novia?

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Qué está a la novia?


In [62]:
greedy_search(model, 'LEONARDO:', 300)

LEONARDO: (A la novia) ¿Qué te pasa?

NOVIO: ¿Qué está a la novia?

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Qué está a la novia?

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Qué está a la novia?

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Qué está a la novia?

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Qué está a la n

In [64]:
# Con texto de la obra
initial_text = 'MOZO 1: (Entrando) ¡Tienes que beber con nosotros!\n\nNOVIO: Estoy esperando a la novia.\n\nMOZO 2: ¡Ya '
print(initial_text)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya 


In [27]:
greedy_search(model, initial_text, N=300)

MOZO 1: (Entrando) ¡Tienes que beber con nosotros!

NOVIO: Estoy esperando a la novia.

MOZO 2: ¡Ya vinte a la cara con el caballo de azahar.)

NOVIA: (Alegre) ¡Qué te puede a la carrel?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente en la cabeza) ¿Qué te puede a la carre?

MADRE: Sí. (Se siente e

In [28]:
greedy_search(model, 'MADRE:', 300)

MADRE: (Seria) Ahora está con la cabeza de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo de la caballo 

$$\huge {\displaystyle P(y=j\mid \mathbf {x} )={\frac {e^{\mathbf {x} ^{\mathsf {T}}\mathbf {w} _{j}/T}}{\sum _{k=1}^{K}e^{\mathbf {x} ^{\mathsf {T}}\mathbf {w} _{k}/T}}}}$$

## Random Sample

In [73]:
def simple_sample(model, phrase, temp=1, N=200):
    print(phrase, end='')
    for i in range(N):
        probs = predict_phrase(model, phrase, verbose=False)
        index, dist = sample_probs(probs, sample_type='simple', temp=temp)
        next_char = indices_to_chars[index]
        phrase=phrase + next_char
        print(next_char, end='')

In [74]:
simple_sample(model, 'NOVIA:', temp=3, N=300)

NOVIA:Vo leétad uncombeja)lplñmLtaCé

RUEGA::
CL s Dpje desutO
A élOsen)
YOnsay
uajrpajerqveblas, Déscáe: Comecpóagoa)
¡u aua?UfOqaí claey casta!., ó aííR
!Uí ñ ñosalópirespihórmanlquecej lacá. Magliamiértras.
¿(cquisR
táf1alee) psgijtern beztaléllis jéabsuilda.
MACRR:dheceséd lajevitnecaim: yl?
ri tueO2L

In [75]:
simple_sample(model, 'JULIAN:', temp=3, N=300)

JULIAN:, Féb,,,
bejo)
CñeGgástaqbu fusdrr,
qued cues.
s blance Bujú. TSuen. geona
.
HijAlfa años
abostaryalA¡goviendoy)
É,
Emcirtoce sebedadilV suc segrompeós,ceslonal.
Cl Cel,asabitaor?.Popenón,crabóondre hes:geréúN(VéDáslmes
elligossp!tibbe!, iz.l josuñ, qeo)nuñasdo isvoeGtla(beeóza.

lEn padés.:Dlá,s.(i

In [76]:
# Cada pasada escribe algo diferente
simple_sample(model, 'NOVIA:', temp=1, N=300)

NOVIA:
Despierte la navia.
CUIADA:

NOVIO: Yo. Yo no soy que sentar de cosa de la mana y la novia en las mujer las manellas. Que no peinacon de ostel y la padre.

CRIADA: (Cogidando de alegres.

MUCHACHA: ¡Al siente) No tengo así.

MADRE: ¿Lo tranes?

MADRE: ¿Lo sí?

NOVIO: ¡Callo que teces?

LEONARDO: De

In [77]:
# Cambiando temperatura
simple_sample(model, 'NOVIA:', 0.1, 300)

NOVIA: (Seria) Ahora es la novia. Pero no se pasa esta con la boda.

NOVIO: ¿Está bien a la novia?

NOVIO: ¿Cuándo es que la casa?

NOVIO: ¿Está contentas?

MADRE: No. Mañana de la novia. Pero no se pasa en la boda.

NOVIO: ¿Está contenta?

NOVIO: ¿A qué me mujer?

NOVIO: ¿Está contente?

NOVIO: ¿Está bue

In [78]:
simple_sample(model, 'NOVIA:', 0.1, 300)

NOVIA: (Seria) ¡Ay!

MADRE: ¿Qué te pasa?

NOVIO: ¿Está contenta?

MADRE: No. Que vengo a la madre.

NOVIO: ¿Qué no sales con el azahar?

NOVIO: ¿Cuándo es que la casa?

MADRE: No. Quiere besar.

NOVIA: ¿Está bien a la novia?

NOVIO: ¿Qué está a la novia?

NOVIO: (A la novia) ¿Qué te pasa?

NOVIO: ¿Está c

In [79]:
simple_sample(model, 'NOVIA:', 2)

NOVIA:SCuaplamultá; loere,
parmela Los arrutas
pamástitas, intéves riontel.
ECEnAO:
¡Desparé
a el quneC? Mira
aelegRi!
del derema
Leorpod que no lejes cocándatasceljojlas sienritmtbia
y du  bado,
Las
pocier

In [80]:
simple_sample(model, initial_text, 0.5, 300)

NameError: name 'initial_text' is not defined

# Stochastic Beam search

https://www.youtube.com/watch?v=zPv7nWB4_ts

In [81]:
def init_models(beam_width):
    models = []
    for i in range(beam_width):
        model = get_deep_rnn((100, 69), dense_units=69, stateful=False, verbose=False)
        model.load_weights('bodes_sangre_LSTM_deep_dropout_04.hdf5')
        models.append(model)
    return models

In [82]:
def predict_beams(phrases, log_probs, beam_width, temp=1):
    samples = []
    log_probs_out = []
    for j, phrase in enumerate(phrases):
        for i in range(beam_width):
            probs = predict_phrase(models[i], phrase, verbose=False)
            index, dist = sample_probs(probs, sample_type='simple', temp=temp)
            samples.append(phrase + indices_to_chars[index])
            log_probs_out.append(log_probs[j] + np.log(dist[index]))
    return np.array(samples), np.array(log_probs_out)

In [84]:
beam_width = 5
models = init_models(beam_width)

In [ ]:
initial_phrase = 'NOVIA:'
phrases = [initial_phrase]
log_probs = np.zeros(len(phrases)*beam_width)
for i in range(100):
    print(phrases, end=' ')
    phrases, log_probs = predict_beams(phrases, log_probs, beam_width, temp=2)
    print('Genero '+str(len(phrases))+' posibles y me quedo con los {} mas probables'.format(beam_width))
    #print(phrases)
    #print(log_probs)
    sorted_indexes = np.argsort(log_probs)[::-1][:beam_width]
    log_probs = log_probs[sorted_indexes]
    phrases = phrases[sorted_indexes]
    print('---------------------------------------------')
print(phrases[0])

['NOVIA:'] WARNING:tensorflow:5 out of the last 4987 calls to <function Model.make_predict_function.<locals>.predict_function at 0x7fd37eeb64c0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
Genero 5 posibles y me quedo con los 5 mas probables
---------------------------------------------
['NOVIA: ' 'NOVIA: ' 'NOVIA:\n' 'NOVIA:D' 'NOVIA:r'] Genero 25 posibles y me quedo con los 5 mas probables
-----------------------

In [41]:
print(phrases[0])

NOVIA: Vamos a la madre. Pero como una mujer.

MADRE: (Al novio) No tengo es un hombre. Pero es un caballo


## Algunas frases con Beam Search
Temp = 1

Temp = 2

NOVIA: Vamos a la madre. Pero como una mujer.

MADRE: (Al novio) No tengo es un hombre. Pero es un caballo

# Verificación Beam search

In [1101]:
selected_pred = 0 # Cual de las predicciones verifico. Deberia quedarme con la de mayor prob
predicted_chars = phrases[selected_pred][len(initial_phrase):]
print(predicted_chars)
log_sum = 0
predicted_phrase_parc = initial_phrase
for c in predicted_chars:
    pred_prob = predict_phrase(model, predicted_phrase_parc, verbose=False)[chars_to_indices[c]]
    print(pred_prob)
    predicted_phrase_parc = predicted_phrase_parc + c
    log_sum = log_sum + np.log(pred_prob)
log_sum

 (Ser
0.525927
0.296634
0.278172
0.86822
0.606687


-3.7784151881933212